In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from collections import Counter
from sklearn.cluster import AffinityPropagation
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
import pandas as pd
from src.utils_clustering import *
import time

In [33]:
N_CLASSES = 15

In [34]:
DATASET_FILE_NAME = "b2"
file_path = rf'..\data\processed\{DATASET_FILE_NAME}.txt'
processed_data = pd.read_csv(file_path)

In [35]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(processed_data)

In [ ]:
# Calculate the preference
# This uses the median of the pairwise distances as a starting point
# You might need to adjust this based on the performance
pairwise_dist = pairwise_distances(processed_data)
preference = np.percentile(pairwise_dist, 50) * -1 
print(preference)

In [ ]:
# Start measuring time
start_time = time.time()

affinity_propagation_clustering = AffinityPropagation(
    preference=preference,
    damping=0.9,  # Adjust damping factor as needed
    max_iter=2000,  # Increased max iterations
    convergence_iter=30,  # Increased convergence iterations for more stability
    random_state=42
)

labels_pred = affinity_propagation_clustering.fit_predict(processed_data)

# Measure elapsed time
end_time = time.time()
running_time = end_time - start_time
running_time_in_seconds = running_time / 60
running_time_in_seconds

In [ ]:
cluster_distribution = Counter(labels_pred)
print("Predicted Distribution: ",cluster_distribution)

In [ ]:
LABELS_FILE_NAME = 's1-label'
labels_true = load_labels_from_file(rf'..\data\label\{LABELS_FILE_NAME}.pa', len(labels_pred))
cluster_distribution = Counter(labels_true)
print(cluster_distribution)

In [ ]:
clustering_algorithm_name = 'AffinityPropagation_clustering'
dataset_name = DATASET_FILE_NAME
results_path = r'../results/metrics/results.csv'

In [ ]:
plot_clusters(processed_data, labels_pred, title=clustering_algorithm_name)

In [ ]:
labels_pred = map_clusters_to_ground_truth(labels_true, labels_pred)

In [ ]:
generate_confusion_matrix(labels_true, labels_pred, N_CLASSES)

In [ ]:
algorithm_params = affinity_propagation_clustering.get_params()
algorithm_details = str(algorithm_params)

In [ ]:
# Evaluate and save results
evaluate_clustering(X=processed_data, labels_true=labels_true, labels_pred=labels_pred,
                    clus_algo_name=clustering_algorithm_name, dataset_name=dataset_name, 
                    results_path=results_path, algorithm_details=algorithm_details, 
                    running_time=running_time)